In [1]:
import numpy as np
import wfdb
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pywt
from pyhrv.hrv import hrv
import pyhrv

In [5]:
def get_signals(path):
    signals = []
    all_meta = []  # To store metadata for each signal
    i = 0
    # Walk through all directories and subdirectories
    for root, dirs, files in os.walk(path):
        for filename in files:
            if filename.endswith('.hea'):
                base_filename = filename[:-4]
                hea_filepath = os.path.join(root, base_filename)
                try:
                    # Read the .hea file using wfdb.rdsamp
                    signal, meta = wfdb.rdsamp(hea_filepath)
                    signals.append(signal)
                    all_meta.append(meta)   
                    i+=1
                except Exception as e:
                    print(f"Failed to read {hea_filepath}: {e}")
    return signals, all_meta

In [55]:
def get_diag(all_meta, csv_file='Nombres.csv'):
    diaglist = []
    df = pd.read_csv(csv_file)
    snomed_cts = df['Snomed_CT'].tolist()
    acronym_names = df['Acronym Name'].tolist()
    full_names = df["Full Name"].tolist()
    #print(acronym_names)
    #print(snomed_cts)
    #print(full_names)
    pairedList = []
    for i in range(len(all_meta)):
      diagnostics = all_meta[i]["comments"][2].split(" ")[1].split(",")
      diaglist.append(diagnostics)
    return diaglist, full_names, snomed_cts
    


In [85]:
def flatten(lista):
    lista_flat = []
    for elemento in lista:
        if isinstance(elemento, list):
            lista_flat.append(elemento[0])
            # lista_flat.extend(flatten(elemento))
        else:
            lista_flat.append(elemento)
    return lista_flat

In [124]:
def freqCount(lista, names, code):
    frecuencias = {}
    for elemento in lista:
        if elemento in frecuencias:
            frecuencias[elemento] += 1
        else:
            frecuencias[elemento] = 1
    frecuencias
    rElements = [elemento for elemento, conteo in frecuencias.items() if conteo > 500]
    rNames = [names[code.index(int(element))] for element in rElements if int(element) in code]
    
    return rNames, rElements

In [116]:
def get_signals(rCodes, meta, signals):
    pairedList = []
    dList = []
    for i in range(len(meta)):
        diagnostics = meta[i]["comments"][2].split(" ")[1].split(",")[0]
        dList.append(diagnostics)
        
    for j in range(0, len(rCodes)):
        iCode = rCodes[j]
        for k in range(0, len(dList)):
            kdList=dList[k]
            if kdList == iCode:
                pairedList.append((signals[k], int(iCode)))
    return pairedList

In [7]:
path =  "C:/Users/Eva/Downloads/a-large-scale-12-lead-electrocardiogram-database-for-arrhythmia-study-1.0.0/a-large-scale-12-lead-electrocardiogram-database-for-arrhythmia-study-1.0.0"
signals, meta = get_signals(path)

Failed to read C:/Users/Eva/Downloads/a-large-scale-12-lead-electrocardiogram-database-for-arrhythmia-study-1.0.0/a-large-scale-12-lead-electrocardiogram-database-for-arrhythmia-study-1.0.0\WFDBRecords\01\019\JS01052: time data '/' does not match format '%d/%m/%Y'
Failed to read C:/Users/Eva/Downloads/a-large-scale-12-lead-electrocardiogram-database-for-arrhythmia-study-1.0.0/a-large-scale-12-lead-electrocardiogram-database-for-arrhythmia-study-1.0.0\WFDBRecords\23\236\JS23074: list index out of range
['1AVB', '2AVB', '2AVB1', '2AVB2', '3AVB', 'ABI', 'ALS', 'APB', 'AQW', 'ARS', 'AVB', 'CCR', 'CR', 'ERV', 'FQRS', 'IDC', 'IVB', 'JEB', 'JPT', 'LBBB', 'LBBBB', 'LFBBB', 'LVH', 'LVQRSAL', 'LVQRSCL', 'LVQRSLL', 'MI', 'MIBW', 'MIFW', 'MILW', 'MISW', 'PRIE', 'PWC', 'QTIE', 'RAH', 'RBBB', 'RVH', 'STDD', 'STE', 'STTC', 'STTU', 'TWC', 'TWO', 'UW', 'VB', 'VEB', 'VFW', 'VPB', 'VPE', 'VET', 'WAVN', 'WPW', 'SB', 'SR', 'AFIB', 'ST', 'AF', 'SA', 'SVT', 'AT', 'AVNRT', 'AVRT', 'SAAWR']
[270492004, 1950420

In [56]:
diag, names, code = get_diag(meta)

In [87]:
flatDiag = flatten(diag)

In [125]:
rNames, rCodes = freqCount(flatDiag, names, code)

In [127]:
rNames

['Atrial Fibrillation',
 'Sinus Bradycardia',
 'Atrial Flutter',
 'Sinus Rhythm',
 'Sinus Tachycardia',
 'T wave Change',
 'Supraventricular Tachycardia',
 'atrial\xa0premature\xa0beats',
 'Sinus Irregularity']

In [126]:
totList = get_signals(rCodes, meta, signals)